In [1]:
#default_exp blocks

Esse código é baseado no `driver.py` do Arthur: https://git.anatel.gov.br/anatel/rfeye
O código contempla as mesmas classes mas com implementação bastante distinta:
* Eliminação de métodos que somente retornam atributos para atributos diretamente.
* Eliminação de Repetição de Código 
* Otimização de Loops para List Comprehensions
* Mudança de Herança para Composição e delegação de atributos da atual para a antiga superclasse, agora como atributo

In [2]:
#export
from datetime import datetime
from collections import namedtuple
from typing import Union, Any, Optional, List, Tuple, Iterator, Mapping, Set, NamedTuple
from rfpy.utils import *
import numpy as np
from fastcore.foundation import GetAttr, L
path_type = Union[str, Any]
bin_val = Union[int, bytes]
bytes_encoded = Union[int, bytes]
datetime_object = datetime

### Block

In [3]:
#export
BASEBLOCK = ['thread_id', 'size', 'type', 'trailer', 'data']

# used in PROC: self._get_processing()
DICT_PROCESSING: Mapping[int, str] = {0: 'single measurement', 1: 'average', 2: 'peak', 3: 'minimum'}
# used in DTYPE: self._get_unit()
DICT_UNIT: Mapping[int, str] = {0: '%', 1: 'dBm', 2: 'dBuV/m'}
    
BaseBlock = namedtuple('BaseBlock', ' '.join(BASEBLOCK))

def create_base_block(bin_block: bytes)->Tuple:
    """
    Um bloco é um pedaço do arquivo .bin com inicio e final conhecidos e que contém diferentes tipos de informação.
    Possui varios campos: tipo, cabeçalho, dados e rodape.
    Cada campo possui comprimentos e informações definidas na documentação.
    Recebe um bloco do arquivo bin e retorna uma instância de Base Block com os atributos
    'thread_id', 'block_size', 'block_type', 'block_trailer', 'block_data'
    """
    if not isinstance(bin_block, bytes):
        raise TypeError(f"Expected type 'bytes', got '{type(bin_block)}' instead.")
    return BaseBlock(*L(bin_block[:4], bin_block[4:8], bin_block[8:12]).map(bin2dec), bin_block[-4:], bin_block[12:])

### TimedBlock

In [4]:
#export
class TimedBlock(GetAttr):
    """
    Aplicável aos tipos de bloco:
        - DataType65
        - DataType63

    Implementa o mapeamento de funções dos blocos que possuem os seguintes campos
    (na mesma posição do vetor binario):F0 a F8

    F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
    F1 = (4 bytes) WALLTIME = Wall Clock Start Time
    F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
    F3 = (2u bytes) STARTMEGA = Start Frequency MHz
    F4 = (4 bytes) STARTMILLI = Start Frequency mHz
    F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
    F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
    F7 = (2u bytes) STARTCHAN = Start Channel number
    F8 = (2u bytes) STOPCHAN = Stop Channel number

    Outros campos em posições diferentes do vetor binario devem possuir métodos na classe herdada:
    NAMAL, ANTUID, PROC, DTYPE, GERROR, GFLAGS, GROUPID, NPAD e NDATA

    """
    #The attributes which don't belong to this class are delegated to default: i.e Block
    def __init__(self, block: NamedTuple):
        self.default = block 
    
    @property
    def date(self): return bin2date(self.data[:4]) #F0
    @property
    def time(self): return bin2time(self.data[4:8]) #F1
    @property
    def nanosecs(self): return bin2dec(self.data[8:12], False) #F2
    @property
    def start_mega(self): return bin2dec(self.data[12:14], False) #F3
    @property
    def stop_mega(self): return bin2dec(self.data[18:20], False) #F5
    @property
    def start_mili(self): return bin2dec(self.data[14:18])
    @property
    def stop_mili(self): return bin2dec(self.data[20:24])
    @property
    def banda_mega(self): return bin2dec(self.data[12:14], False), bin2dec(self.data[18:20], False) #F3 e F5
    @property
    def banda_mili(self): return bin2dec(self.data[14:18]), bin2dec(self.data[20:24]) #F4 e F6
    @property
    def channel(self): return bin2dec(self.data[24:26], False), bin2dec(self.data[26:28], False) #F7 e F8
    @property
    def datetime_stamp(self):
        return  datetime(2000+self.date[2], self.date[1], self.date[0],
                            self.time[0], self.time[1], self.time[2], int(self.nanosecs/1000))

### Data Type 21 – Unit and Job Information

In [5]:
#export
class DType21(GetAttr):
    """
    Data Type 21 – Unit and Job Information
    Tipo de bloco existente a partir da versão 3 do script LOGGER.
    F0 = (16 bytes) HOSTNAME = Unit Hostname.
                               Null terminated string (up to length 15), padded with nulls.
    F1 = (4 bytes) TEXT1LEN = Free Text length.
                              Including null termination and padding (must be a whole number of 4 bytes).
    F2 = (TEXT1LEN) TEXT1 = Null terminated Free Text – unit_info.
                            From ‘unit_info’ in cfg file Padded with \0 bytes,
                            E.g. could include
                                ‘method=blah;
                                other_key=blerg’ etc
                            keyword lookup in later analysis.
    F3 = (4 bytes) TEXT2LEN = Free Text length.
                              Including null termination and padding (must be a whole number of 4 bytes).
    F4 = (TEXT4LEN) TEXT4 = Null terminated Free Text – method.
                            From ‘method’ in cfg file.
    """
    #The attributes which don't belong to this class are delegated to default: i.e Block
    def __init__(self, block: NamedTuple):
        self.default = block 
        
    def _get_text1_len(self) -> int:
        """
        Retorna o tamanho do campo TEXT1 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        return bin2dec(self.data[16:20])
    
    def _get_text2_len(self) -> int:
        """
        Retorna o tamanho do campo TEXT2 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        inicio = 20 + self._get_text1_len()
        final = 4 + inicio
        return bin2dec(self.data[inicio:final])
    
    @property
    def hostname(self) -> str:
        """
        Retorna o campo HOSTNAME que contém o "Unit Hostname".
        :return:
        """
        return bin2str(self.data[:16])

    @property
    def unit_info(self) -> str:
        """
        Retorna o campo TEXT1 que contém o ‘unit_info’ no arquivo cfg.
        :return:
        """
        fim = 20 + self._get_text1_len()
        return bin2str(self.data[16:fim])

    @property
    def method(self) -> str:
        """
        Retorna o campo TEXT2 que contém o ‘method’ no arquivo cfg.
        :return:
        """
        inicio = 24 + self._get_text1_len()
        fim = inicio + self._get_text2_len()
        return bin2str(self.data[inicio:fim])

### Data Type 24 – Data Thread Information

In [6]:
#export
class DType24(GetAttr):
    """
    Data Type 24 – Data Thread Information.
    Tipo de bloco existente a partir da versão 4 do script LOGGER.

    F0 = (4 bytes) GROUPID = ID used to group sets of data
                             0 = not a member of a group
    F1 = (4 bytes) TEXTLEN = Free Text Length
    F2 = (TEXTLEN) TEXT = Null terminated Free Text
                    description of the measurement thread - desc
    """
     #The attributes which don't belong to this class are delegated to default: i.e Block
    def __init__(self, block: NamedTuple):
        self.default = block 
        
    @property
    def group_id(self) -> int:
        """
        self > int
        :return: Número identificador do grupo. Zero significa que não faz parte de nenhum grupo.

        F0 = (4 bytes) GROUPID = ID used to group sets of data
                             0 = not a member of a group
        """
        return bin2dec(self.data[:4])

    @property
    def len_text(self) -> int:
        """
        self > int
        :return: Comprimento do texto incluindo bytes nulos ao final (número inteiro de 4 bytes).

        F1 = (4 bytes) TEXTLEN = Free Text Length
        """
        return bin2dec(self.data[4:8])

    @property
    def text(self) -> str:
        """
        self > str
        :return: Texto armazenado no bloco sem os bytes nulos ao final.

        F2 = (TEXTLEN) TEXT = Null terminated Free Text
                    description of the measurement thread - desc
        """
        return bin2str(self.data[8:])

### Data Type 40 – GPS Data

In [7]:
#export
class DType40(GetAttr):
    """
    O Bloco do tipo 40 carrega dados de GPS.

    Data Type40 - GPS Data
    """
    
    def __init__(self, block: NamedTuple):
        """This implementation substitues inheritance of the class Block by Composition
        The attributes which belong to Block are accessed normally as if it was Inherited
        """        
        self.default = block         
    
    @property
    def date(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: dia, mês, ano, reserva

        F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
        """
        return bin2date(self.data[:4])

    @property
    def time(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: horas, minutos, segundos, décimos de segundo

        F1 = (4 bytes) WALLTIME = Wall Clock Start Time
        """
        return bin2time(self.data[4:8])
    
    @property
    def nanosecs(self) -> int:
        """
        self > int
        :return: nanosegundos

        F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
        """
        return bin2dec(self.data[8:12], False)
    
    @property
    def wallclock_date_time_stamp(self) -> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return self.date, self.time, self.nanosecs
    
    @property
    def gps_date(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: dia, mês, ano, reserva

        F3 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
        """
        return bin2date(self.data[12:16])

    @property
    def gps_time(self) -> Tuple[int, int, int, int]:
        """
        self > int, int, int, int
        :return: horas, minutos, segundos, décimos de segundo

        F4 = (4 bytes) WALLTIME = Wall Clock Start Time
        """
        return bin2time(self.data[16:20])

    @property
    def gps_date_time_stamp(self)-> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return self.gps_date, self.gps_time

    @property
    def gps_status(self) -> Union[str, int]:
        """
        self > int
        :return: "status" do GPS conforme documentação.

        F5 = (1 byte) Posicional Fix and Status.

        if status = 1:
            0 = No Fix
            1 = Standard GPS
            2 = Differencial GPS.
        if status = 0:
            Set to zero.
        """
        status_dict = {0:'No Fix', 1:'Standard GPS', 2:'Differential GPS'}
        status = bin2dec(self.data[20:21])
        return status_dict.get(status, status)

    @property
    def num_sattelites(self) -> int:
        """
        self > int
        :return: Número de satélites em visada.

        F6 = (1 byte) Sattelites in view

        0 = bad, 1+ better
        """
        return bin2dec(self.data[21:22])

    @property
    def heading(self) -> float:
        """
        self > float
        :return: Azimute de direção.

        F7 = (2 bytes) Heading = Degrees * 100
        """
        return bin2dec(self.data[22:24])/100

    @property
    def latitude(self) -> float:
        """
        self > float
        :return: Latitude em graus decimais. Valores positivos para emisferio Norte e negativos para Sul.

        F8 = (4 bytes) Latitude = Degrees * 1000000: +ve=N, -ve=S
        """
        return bin2dec(self.data[24:28]) / 1000000

    @property
    def longitude(self) -> float:
        """
        self > float
        :return: Longitude em graus decimais. Valores positivos para Leste e negativos para Oeste de greenwich.

        F9 = (4 bytes) Longitude = Degrees * 1000000: +ve=E, -ve=W
        """
        return bin2dec(self.data[28:32]) / 1000000

    @property
    def speed(self) -> float:
        """
        self > float
        :return: Velocidade em quilometros por hora.

        F10 = (4 bytes) Speed = kph * 1000
        """
        return bin2dec(self.data[32:36]) / 1000

    @property
    def altitude(self) -> float:
        """
        :return: Altidude do nivel do mar em metros.

        self > float

        F11 = (4 bytes) Altitude = Metres * 1000
        """
        return bin2dec(self.data[36:40]) / 1000

### Data Type 41 – Timed Free Text Information

In [8]:
#export
class DType41(GetAttr):
    """
    O Bloco do tipo 41 é uma informação em texto livre com marca temporal
    O tamanho total em bytes é (48 + NTEXT)

    F0 = (4 bytes) Date from Wall Clock (dd/mm/yy/null)
    F1 = (4 bytes) Time from Wall Clock (hh/mm/ss/cc)
    F2 = (4 bytes - 32 bit unsigned integer) Wall Clock Start Time Nanoseconds. Must be 0 if cc>0.
    F3 = (32 bytes) Fixed Length Data Type Identifier, null terminated string
        Tells the software how to deal with the free text.
        Defined types:
        - LOGGER_NAME: Application name
        - LOGGER_VERSION: Application version
        - AUDIT: Audit process output
        - GPRS: GPRS message
        - GSM: Cell survey information
        - INFO: Voltages, currents and temperatures
        - LED: LED status
        - MASK: Mask status
        - MESSAGE: Message text
        - NMEA: GSM NMEA text
        - SNMP: SNMP Message text
        - CONF: NCPD Configuration
    F4 = (4 bytes) NTEXT = Free Text Length Including null termination and padding (must be a whole number of 4 bytes)
    F5 = (NTEXT bytes) Null terminated Free Text
    """
    def __init__(self, block: NamedTuple):
        """This implementation substitues inheritance of the class Block by Composition
        The attributes which belong to Block are accessed normally as if it was Inherited
        """        
        self.default = block       
    
    @property
    def date(self) -> Tuple[int, int, int, int]:
        """
        self> int,int,int,int
        :return: dia, mês, ano, reserva

        F0 = (4 bytes) Date from Wall Clock (dd/mm/yy/null)
        """
        return bin2date(self.data[:4])

    def time(self) -> Tuple[int, int, int, int]:
        """
        self> int,int,int,int
        :return: horas, minutos, segundos, décimos de segundo

        F1 = (4 bytes) Time from Wall Clock (hh/mm/ss/cc)
        """
        return bin2time(self.data[4:8])

    def nanosecs(self) -> int:
        """
        self> int
        :return: nanosegundos

        F2 = (4 bytes - 32 bit unsigned integer) Wall Clock Start Time Nanoseconds.
                    Must be 0 if cc>0.
        """
        return bin2dec(self.data[8:12], False),

    @property
    def date_time_stamp(self) -> Tuple[Tuple[int, int, int, int], Tuple[int, int, int, int], int]:
        """
        self > int, int, int, int, int, int, int, int
        :return: dia, mês, ano, reserva, horas, minutos, segundos, décimos de segundo
        """
        return self.date, self.time, self.nanosecs

    @property
    def identifier(self) -> str:
        """
        self > str
        :return: Identificador Descritivo (texto).

        F3 = (32 bytes) Fixed Length Data Type Identifier, null terminated string
        Tells the software how to deal with the free text.
        Defined types:
        - LOGGER_NAME: Application name
        - LOGGER_VERSION: Application version
        - AUDIT: Audit process output
        - GPRS: GPRS message
        - GSM: Cell survey information
        - INFO: Voltages, currents and temperatures
        - LED: LED status
        - MASK: Mask status
        - MESSAGE: Message text
        - NMEA: GSM NMEA text
        - SNMP: SNMP Message text
        - CONF: NCPD Configuration
        """
        return bin2str(self.data[12:44])

    @property
    def len_text(self) -> int:
        """
        self > int
        :return: Número de caracteres do texto.

        F4 = (4 bytes) NTEXT = Free Text Length
                    Including null termination and padding
                    (must be a whole number of 4 bytes)
        """
        return bin2dec(self.data[44:48])

    @property
    def text(self) -> str:
        """
        self > str
        :return: O texto armazenado no bloco.
        F5 = (NTEXT bytes) Null terminated Free Text
        """
        return bin2str(self.data[48:])

### Data Type 63 – Spectral Data

In [9]:
#export
class DType63(GetAttr):
    """
    O Bloco do tipo 63 carrega dados de Nível por Canal de Frequência.

    Data Type 63 – Spectral Data

    F0 a F8, F10 a F13, F15 a F17 e F20 a F21 já inicializados na classe TimedBlock.

   *F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
   *F1 = (4 bytes) WALLTIME = Wall Clock Start Time
   *F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
   *F3 = (2u bytes) STARTMEGA = Start Frequency MHz
   *F4 = (4 bytes) STARTMILLI = Start Frequency mHz
   *F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
   *F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
   *F7 = (2u bytes) STARTCHAN = Start Channel number
   *F8 = (2u bytes) STOPCHAN = Stop Channel number
    F9 = (4 bytes) SAMPLE = Duration of sampling.
                    Time taken by the FPGA and Radio to execute command in µs
    F10 =(4 bytes) NAMAL = Amalgamated  Results
    F11 = (1u byte) ANTUID Antenna number [ 0- 255]
    F12 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
    F13 = (1 byte) DTYPE = Data Type
                    1 = dBm,
                    2 = dBuV/m
    F14 = (1 byte) OFFSET = Data level offset in DTYPE units
                    2’s Complement, range [-128, 127].
    F15 = (1 byte)
    F16 = (1 byte)
    F17 = (1 byte)
    F18 = (2 bytes)
    F19 = (2 bytes)
    F20 = (1 byte)
    F21 = (4 bytes)
    F22 = (4*NTUN bytes)
    F23 = (NAGC bytes)
    F24 = (NDATA bytes)
    F25 = (NPAD bytes)
    """    
    def __init__(self, block: NamedTuple)-> None:
        self.default = TimedBlock(block)
        
    @property
    def spent_time_microsecs(self) -> int:
        """
        self > int
        :return: O tempo levado pelo radio e FPGA para a aquisição da amostra, em microsegundos.

        F9 = (4 bytes) SAMPLE = Duration of sampling.
                    Time taken by the FPGA and Radio to execute command in µs
        """
        return bin2dec(self.data[28:32])

    @property
    def num_meas(self) -> int:
        """
        self > int
        :return: O número de resultados agrupados. Se for 1 equivale a uma única medida.

        Overrides method in TimedBlock.

        F10 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
        """
        return bin2dec(self.data[32:36])

    @property
    def id_antenna(self) -> int:
        """
        self > int
        :return: o ID da antena usada na medida.

        Overrides method in TimedBlock.

        F11 = (1u bytes) ANTUID Antenna number [ 0- 255]
        """
        return bin2dec(self.data[32:33], False)

    @property
    def processing(self) -> Union[str, int]:
        """
        self > (int, str)
        :return: O código e a descrição do tipo de processamento aplicado à medida.

        Overrides method in TimedBlock.

        F12 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
        """
        proc = bin2dec(self.data[37:38])
        return DICT_PROCESSING.get(proc, proc)

    @property
    def unit(self) -> Union[str, int]:
        """
        self > str or int
        :return: A unidade de medida.
                Retorna um número inteiro se a unidade não foi documentada.

        Overrides method in TimedBlock.

        todo: Não está funcionando, todos os dados retornam o valor 0!

        F13 = DTYPE = Data Type
            0 = % (não documentado)
            1 = dBm
            2 = dBuV/m
        """
        unit: int = bin2dec(self.data[38:39])
        return DICT_UNIT.get(unit, unit)

    @property
    def level_offset(self) -> int:
        """
        self > int
        :return: O deslocamento em nível adotado na compactação dos dados binários.
                 A unidade é dada por self._get_unit().

        F14 = (1 bytes) OFFSET = Data level offset in DTYPE units
                                2’s Complement, range [-128, 127].
        """
        return bin2dec(self.data[39:40])

    @property
    def global_error_code(self) -> int:
        """
        self > int
        :return: O código de erro global.

        Overrides method in TimedBlock.

        F15 = (1 byte) GERROR = Global Error Code.
                    Radio or processing global error code

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.data[40:41])

    @property
    def global_flags_code(self) -> int:
        """
        self > int
        :return: Códigos de alertas globais ou de processamento do radio.

        Overrides method in TimedBlock.

        F16 = (1 byte) GFLAGS = Global clipping flags etc.
                    Radio or processing global flags.

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.data[41:42])

    @property
    def group_id(self) -> int:
        """
        self > int
        :return: O ID do grupo à qual a medida pertence.
                 Caso 0 não pertence a nenhum grupo.
                 Use a classe 'DataType24' para detalhes do grupo.

        Overrides method in TimedBlock.

        F17 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
        """
        return bin2dec(self.data[42:43])

    @property
    def len_tunning_info(self) -> int:
        """
        self > int
        :return: 0 ou igual à quantidade de valores de AGC usados na amostra.

        F18 = (2 bytes) NTUN= Number of 4 byte Tuning info blocks. 0 or NAGC
        """
        return bin2dec(self.data[43:45])

    @property
    def len_agc(self) -> int:
        """
        self > int
        :return: 0 ou igual à quantidade de valores de "tunings" usados na amostra.

        F19 = (2 bytes) NAGC = Number of single byte AGC values
                                Can be 0, or = number of tunings in the sweep
        """
        return bin2dec(self.data[45:47])

    @property
    def len_padding(self) -> int:
        """
        self > int
        :return: Valor que varia de 0 a 3 indicando o preenchimento nulo para manter o tamanho do bloco (em bytes) fixo.

        Overrides method in TimedBlock.

        A extração do NPAD (quantidade) tem pouca utilidade prática.
        A extração do Padding (valor) não tem utilidade prática.

        F20 = (1 byte) NPAD = Number of bytes of padding. 0-3
        (NPAD bytes) Padding = As \0 bytes
        """
        return bin2dec(self.data[39:40])

    @property
    def data_points(self) -> int:
        """
        self > int
        :return: O número de canais (ou "steps") que dividem igualmente a largura de banda .

        F21 = (4 bytes) NDATA = Number of single byte data points.
                        Number of equal width channels dividing the reported frequency width
        """
        return bin2dec(self.data[48:52])
       
    @property
    def bw(self):
        return self.banda_mega[1] - self.banda_mega[0]
    
    @property
    def passo(self):
        return self.bw / (self.data_points - 1)
    
#     @property
#     def list_freq_mega(self) -> np.array:
#         return np.asarray([self[i] for i in range(len(self))], dtype=np.float32)
    
    @property
    def list_freq_mega(self) -> L:
        return L(*self)


    @property
    def rbw(self) -> float:
        """
        Retorna o RBW calculado a partir de STARTMEGA, STOPMEGA e NDATA.
        :return: RBW (Hz)
        """
        return int(self.passo * 1000000)

    @property
    def tunning_info_array(self) -> bytes:
        """
        self > bytes
        :return: Binario bruto de informações do 'tunning'.

        F22 = (4*NTUN bytes) = Array of Tuning info blocks
                        One block per tuning (1 or 10 MHz). See section 11.7.

        todo: implementar um método para traduzir o 'Tuning info block' de acordo com a seção 11.7 da documentação.
        """
        start = 52  # inicia aqui
        stop = start+(self.len_tunning_info*4)  # termina aqui
        return self.data[start:stop]

    @property
    def agc_array(self) -> bytes:
        """
        self > bytes
        :return: Binario bruto com toda a matriz de AGC.

        F23 = (NAGC  bytes) = Array of AGC values As dB in single unsigned byte.
                        Current actual values are 0..63.

        todo: implementar um método que forneça a matriz de números inteiros do AGC.
        """
        start = 52+(self.len_tunning_info * 4)  # inicia após F22
        stop = start+(self.len_agc)  # termina aqui
        return self.data[start:stop]

    @property
    def block_data(self) -> List[float]:
        """
        self > List[float]
        :return: A lista de todas as medidas de espectro do bloco em 'dB'.

        F24 = (NDATA  bytes) = Array of dB spectrum data points.
        Data points in a byte representing  the power in dBm in 0.5 dBm steps
        """
        start = 52+(self.len_tunning_info * 4) + self.len_agc  # inicia após F23
        stop = start+(self.data_points)  # termina aqui
        return np.fromiter(self.data[start:stop], dtype=np.float16) / 2 + self.level_offset - 127.5
    
    def __getitem__(self, i):
        """
        self, int > float
        :param index: Número da posição do canal (ou 'step') desejado.
        :return: O valor em MHz do início do canal (ou 'step').
        i é a posição de certo valor em "DataType65._get_data()
        Substitutes the old method point_freq_mega and make the class indexable
        """
        return self.banda_mega[0] + i*self.passo
    
    def __len__(self):
        return int(self.data_points)
    
    def __iter__(self):
        return (self[i] for i in range(len(self)))

### Data Type 65 – Frequency Channel Occupancy (ITU-R SM.1880)

In [10]:
#export
class DType65(GetAttr):
    """
    O Bloco do tipo 65 carrega dados de Taxa de Ocupação Espectral por Canal de Frequência (ITU-R SM.1880).

    O tamanho total em bytes é (48 + NDATA + NPAD)

    F0 a F16 e F19 já inicializados na classe TimedBlock.

   *F0 = (4 bytes) WALLDATE = Wall Clock Start Date of measurements
   *F1 = (4 bytes) WALLTIME = Wall Clock Start Time
   *F2 = (4u bytes) WALLNANO = Wall Clock Start Time Nanoseconds
   *F3 = (2u bytes) STARTMEGA = Start Frequency MHz
   *F4 = (4 bytes) STARTMILLI = Start Frequency mHz
   *F5 = (2u bytes) STOPMEGA = Stop Frequency MHz
   *F6 = (4 bytes) STOPMILLI = Stop Frequency mHz
   *F7 = (2u bytes) STARTCHAN = Start Channel number
   *F8 = (2u bytes) STOPCHAN = Stop Channel number
    F9 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
    F10 = (1u bytes) ANTUID Antenna number [ 0- 255]
    F11 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum
    F12 = (2 bytes) DTYPE = Data Type
                    1 = dBm,
                    2 = dBuV/m
    F13 = (1 byte) GERROR = Global Error Code. Radio or processing global error code
    F14 = (1 byte) GFLAGS = Global clipping flags etc. Radio or processing global flags:
    F15 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
    F16 = (1 byte) NPAD = Number of bytes of padding. 0-3
    F17 = (2 bytes) THRESH = Threshold Level in DTYPE
    F18 = (2 bytes) DURATION = Duration of sampling. In seconds  (i.e. 300, 900, 1800, 2600)
    F19 = (4 bytes) NDATA = Number of single byte data points.
                    Number of equal width channels dividing the reported frequency width
    F20 = (NDATAu bytes) Array of data points.
                    Each data point is stored as a single byte number
                    representing  the percentage (0..100 % in 0.5 steps)
    F21 = (NPAD bytes) Padding. As \0 bytes
    """
    def __init__(self, block: NamedTuple)-> None:
        self.default = TimedBlock(block)
        
    @property
    def num_meas(self) -> int:
        """
        self > int
        :return: O número de resultados agrupados. Se for 1 equivale a uma única medida.

        Overrides method in TimedBlock.

        F9 = (4 bytes) NAMAL = Amalgamated Results
                    i.e. ‘number of loops’. Equal to 1 if single measurement.
        """
        return bin2dec(self.data[28:32])

    @property
    def antenna(self) -> int:
        """
        self > int
        :return: o ID da antena usada na medida.

        Overrides method in TimedBlock.

        F10 = (1u bytes) ANTUID Antenna number [ 0- 255]
        """
        return bin2dec(self.data[32:33], False)

    @property
    def processing(self) -> Union[str, int]:
        """
        self > (int, str)
        :return: O código e a descrição do tipo de processamento aplicado à medida.

        Overrides method in TimedBlock.

        F11 = (1 byte) PROC = Processing
                    0 = single measurement,
                    1 = average,
                    2 = peak,
                    3 = minimum

        Data Type 65, Field 11
        Data Type 63, Field 12
        """
        proc = bin2dec(self.data[33:34])
        return DICT_PROCESSING.get(proc, proc)

    @property
    def unit(self) -> Union[str, int]:
        """
        self > str or int
        :return: A unidade de medida.
                Retorna um número inteiro se a unidade não foi documentada.

        Overrides method in TimedBlock.

        todo: Não está funcionando, todos os dados retornam o valor 0!

        F12 = (1 byte) DTYPE = Data Type
            0 = % (não documentado)
            1 = dBm
            2 = dBuV/m
        """
        unit = bin2dec(self.data[34:36])
        return DICT_UNIT.get(unit, unit)

    @property
    def global_error_code(self) -> int:
        """
        self > int
        :return: O código de erro global.

        Overrides method in TimedBlock.

        F13 = (1 byte) GERROR = Global Error Code.
                    Radio or processing global error code

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.data[36:37])

    @property
    def global_flags_code(self) -> int:
        """
        self > int
        :return: Códigos de alertas globais ou de processamento do radio.

        Overrides method in TimedBlock.

        F14 = (1 byte) GFLAGS = Global clipping flags etc.
                    Radio or processing global flags.

        The radio error codes and flags can be provided on request.
        """
        return bin2dec(self.data[37:38])

    @property
    def group_id(self) -> int:
        """
        self > int
        :return: O ID do grupo à qual a medida pertence.
                 Caso 0 não pertence a nenhum grupo.
                 Use a classe 'DataType24' para detalhes do grupo.

        Overrides method in TimedBlock.

        F15 = (1 bytes) GROUPID = ID used to group sets of data
                    0 = not a member of a group
        """
        return bin2dec(self.data[38:39])

    @property
    def len_padding(self) -> int:
        """
        self > int
        :return: Valor que varia de 0 a 3 indicando o preenchimento nulo para manter o tamanho do bloco (em bytes) fixo.

        Overrides method in TimedBlock.

        A extração do NPAD (quantidade) tem pouca utilidade prática.
        A extração do Padding (valor) não tem utilidade prática.

        F16 = (1 byte) NPAD = Number of bytes of padding. 0-3
        (NPAD bytes) Padding = As \0 bytes
        """
        return bin2dec(self.data[47:48])

    @property
    def threshold(self) -> int:
        """
        self > int
        :return: O valor do limiar de detecção. A unidade é dada por self._get_unit().

        F17 = (2 bytes) THRESH = Threshold Level in DTYPE
        """
        return bin2dec(self.data[40:42])

    @property
    def duration_seconds(self) -> int:
        """
        self > int
        :return: A duração da amostra em segundos.

        F18 = (2 bytes) DURATION = Duration of sampling. In seconds
                    (i.e. 300, 900, 1800, 2600)
        """
        return bin2dec(self.data[42:44])

    @property
    def data_points(self) -> int:
        """
        self > int
        :return: O número de canais (ou "steps") que dividem igualmente a largura de banda .

        Overrides method in TimedBlock.

        F19 = (4 bytes) NDATA = Number of single byte data points.
                        Number of equal width channels dividing the reported frequency width
        """
        return bin2dec(self.data[44:48])

    def block_data(self) -> List[float]:
        """
        self > -> List[float]
        :return: A lista de todas as medidas de ocupação do bloco em '%'.

        F20 = (NDATAu bytes) Array of data points.
                    Each data point is stored as a single byte number
                    representing  the percentage (0..100 % in 0.5 steps)
        """
        start = 48
        end = self.data_points
        return np.fromiter(self.data[start:end], dtype=np.float16) / 2

TYPE_CLASS = {
    1413563424: None,
    21: DType21,
    24: DType24,
    40: DType40,
    41: DType41,
    63: DType63,
    65: DType65
}

def block_constructor(btype, bloco):
    constructor = TYPE_CLASS.get(btype)
    if constructor:
        return constructor(bloco)
    return None

In [11]:
from nbdev.export import notebook2script
notebook2script()

C:\Users\rsilva\Miniconda3\envs\bin\lib\site-packages\fastprogress\fastprogress.py:102: UserWarning: Couldn't import ipywidgets properly, progress bar will use console behavior
  warn("Couldn't import ipywidgets properly, progress bar will use console behavior")


Converted 00_main.ipynb.
Converted 01_parser.ipynb.
Converted 02_utils.ipynb.
Converted 03_test_run.ipynb.
Converted 04_blocks.ipynb.
Converted index.ipynb.
